In [4]:
'''
Capstone Project on segmentation and clustering of neighbor hoods in Toroto
'''
#!pip install wikipedia
#!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge geopy --yes
#!pip install beautifulsoup4
#!pip install lxml

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import wikipedia as wp

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#Map rendering
import folium

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from bs4 import BeautifulSoup
from IPython.display import display_html
print("Imported Libraries")

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_source = requests.get(URL).text
soup = BeautifulSoup(wiki_source,'lxml')

wiki_table = str(soup.table)

#display_html(wiki_table,raw=True)

wiki_df = pd.read_html(wiki_table)[0]

df = wiki_df[wiki_df.Borough != 'Not assigned' ]

df_group = df.groupby(['Postcode','Borough'],sort=False).agg(','.join)
df_group.reset_index(inplace=True)

for index,row in df_group.iterrows():
    if row['Neighbourhood']== 'Not assigned':
        row['Neighbourhood'] = row['Borough']
print('CSV Grouping Completed')

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------

In [5]:
df_group.shape

(103, 3)

In [6]:
df_group

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [10]:
'''
Question 2 : Get lat long values
'''
#FILE = 'Geospatial_Coordinates.csv'
#df_group_lat_long = pd.read_csv(FILE)
df_group_lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
df_group_lat_long.rename(columns={'Postal Code':'Postcode'},inplace = True)
df_group_lat_long.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_data_merge = pd.merge(df_group_lat_long,df_group,on='Postcode')
df_data_merge.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge,Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
